# GraphRAG Python package
End-to-end-example on research papers. 

In [ ]:
%%capture
%pip install fsspec langchain-text-splitters openai python-dotenv numpy torch

In [ ]:
%%capture
%pip install -U git+https://github.com/neo4j/neo4j-graphrag-python

In [ ]:
from dotenv import load_dotenv
import os

# load neo4j credentials (and openai api key in background)
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')

## Knowledge Graph Building


In [ ]:
import neo4j
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

driver = neo4j.GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))

llm=OpenAILLM(
    model_name="gpt-4o-mini",
    model_params={
        "response_format": {"type": "json_object"}, # use json_object formatting for best results
        "temperature": 0 # turning temperature down for more deterministic results
    }
)

#create text embedder
embedder = OpenAIEmbeddings()

In [ ]:
#define node labels
basic_node_labels = ["Object", "Entity", "Group", "Person", "Organization", "Place"]

academic_node_labels = ["ArticleOrPaper", "PublicationOrJournal"]

medical_node_labels = ["Anatomy", "BiologicalProcess", "Cell", "CellularComponent", 
                       "CellType", "Condition", "Disease", "Drug",
                       "EffectOrPhenotype", "Exposure", "GeneOrProtein", "Molecule",
                       "MolecularFunction", "Pathway"]

node_labels = basic_node_labels + academic_node_labels + medical_node_labels

# define relationship types
rel_types = ["ACTIVATES", "AFFECTS", "ASSESSES", "ASSOCIATED_WITH", "AUTHORED",
             "CAUSES", "CITES", "CLASSIFIES", "COLLABORATES_WITH", "CONTRIBUTES_TO",
             "CORRELATES_WITH", "DESCRIBES", "DEVELOPED", "DISCUSSES", "EXHIBITS",
             "EXPRESSES", "HAS_EFFECT", "HAS_SYMPTOM", "INCLUDES", "INDUCES",
             "INTERACTS_WITH", "INVOLVES", "LEADS_TO", "LINKED_TO", "LOCATED_IN",
             "MANIFESTS_AS", "OBSERVED_IN", "PARTICIPATES_IN", "PART_OF", "PRODUCES",
             "PUBLISHED_IN", "REACTS_WITH", "REDUCES", "RELATED_TO", "RESULTS_IN",
             "TARGETS", "TREATMENT_FOR", "TRIGGERS", "USED_FOR",
             "USED_WITH", "USES"]


In [ ]:
prompt_template = '''
You are a medical researcher tasks with extracting information from papers 
and structuring it in a property graph to inform further medical and research Q&A

Extract the entities (nodes) and specify their type from the following Input text.
Also extract the relationships between these nodes. the relationship direction goes from the start node to the end node. 


Return result as JSON using the following format:
{{"nodes": [ {{"id": "0", "label": "the type of entity", "properties": {{"name": "name of entity", "details":" brief description of entity (dont include info about relationships)" }} }}],
  "relationships": [{{"type": "TYPE_OF_RELATIONSHIP", "start_node_id": "0", "end_node_id": "1", "properties": {{"details": "brief description of relationship if needed"}} }}] }}

- Use only the information from the Input text.  Do not add any additional information.  
- If the input text is empty, return empty Json. 
- Make sure to create as many nodes and relationships as needed to offer rich medical context for further research.
- An AI knowledge assistant must be able to read this graph and immediately understand the context to inform detailed research questions. 
- Multiple documents will be ingested from different sources and we are using this property graph to connect information, so make sure entity types are fairly general. 

Use only fhe following nodes and relationships (if provided):
{schema}

Assign a unique ID (string) to each node, and reuse it to define relationships.
Do respect the source and target node types for relationship and
the relationship direction.

Do not return any additional information other than the JSON in it.

Examples:
{examples}

Input text:

{text}
'''

In [ ]:
from neo4j_graphrag.experimental.pipeline.kg_builder import SimpleKGPipeline

kg_builder_pdf = SimpleKGPipeline(
    llm=llm,
    driver=driver,
    embedder=embedder,
    entities=node_labels,
    relations=rel_types,
    from_pdf=True
)

In [ ]:
pdf_file_paths = ['truncated-pdfs/biomolecules-11-00928-v2-trunc.pdf', 
             'truncated-pdfs/GAP-between-patients-and-clinicians_2023_Best-Practice-trunc.pdf', 
             'truncated-pdfs/pgpm-13-39-trunc.pdf']

for path in pdf_file_paths:
    print(f"Processing : {path}")
    pdf_result = await kg_builder_pdf.run_async(file_path=path)
    print(f"PDF Processing Result: {pdf_result}")

## Knowledge Graph Retrieval

We will leverage Neo4j's vector search capabilities here. To do this we need to begin by creating a vector index on the text in our Chunk nodes

In [ ]:
from neo4j_graphrag.indexes import create_vector_index

create_vector_index(driver, name="text_embeddings", label="Chunk",
                    embedding_property="embedding", dimensions=1536, similarity_fn="cosine")

Now that the index is set up we will start simple with a VectorRetriever.  The VectorRetriever just queries Chunk nodes, brining back the text and some metadata

In [ ]:
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    driver,
    index_name="text_embeddings",
    embedder=embedder,
    return_properties=["text"],
)

In [ ]:
vector_res = vector_retriever.get_search_results(query_text = "How is precision medicine applied to Lupus?", 
                                                 top_k=3)
for i in vector_res.records: print("====\n" + '...' + i.data()['node']['text'][500:700] + '...')

The GraphRAG Python Package offers a whole host of other useful retrieval covering different patterns.

Below we will use the VectorCypherRetriever which allows you to run a graph traversal after finding text chunks.  We will use the Cypher Query language to define the logic to traverse the graph.  

As a simple starting point, lets traverse up to 2 hops out from each chunk and textualize the different relationships we pick up.  We will use something called a quantified path pattern to accomplish in this. 

In [ ]:
from neo4j_graphrag.retrievers import VectorCypherRetriever

vc_retriever = VectorCypherRetriever(
    driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
//node = Chunk.  Go out 2-3 hops in the entity graph 
MATCH (node)<-[:FROM_CHUNK]-()-[rl:!FROM_CHUNK]-{1,2}()
UNWIND rl AS r
WITH DISTINCT r

//Get the source document(s) from which each entity set was extracted (could be multiple)
MATCH (sourceDoc:Document)<-[FROM_DOCUMENT]-()<-[:FROM_CHUNK]-(n)-[r]->(m)
WITH n,r,m, apoc.text.join(collect(DISTINCT sourceDoc.path), ', ') AS sources

// return textualize relations/triples
RETURN n.name + '(' + coalesce(n.details, '') + ')'+
   ' - ' + type(r) + '(' + coalesce(r.details, '') + ')' +  ' -> ' +
   m.name + '(' + coalesce(m.details, '') + ')' + ' [sourced from: ' + sources + ']' AS fact
"""
)



In [ ]:
vc_res = vc_retriever.get_search_results(query_text = "How is precision medicine applied to Lupus?")

print(f"Retrieved {len(vc_res.records)} records. Previewing the first 3:\n")
for i in vc_res.records[:3]: print("====\n" + i.data()['fact']) 

## GraphRAG 
 You can construct GraphRAG pipelines with the `GraphRAG` class.  At minimum, you will need to pass the constructor an LLM and a retriever. You can also pass a custom prompt template, but for now we will just use the default.
 

In [ ]:

from neo4j_graphrag.llm import OpenAILLM as LLM
from neo4j_graphrag.generation.graphrag import GraphRAG

llm = LLM(model_name="gpt-4o",  model_params={"temperature": 0.0})

v_rag  = GraphRAG(llm=llm, retriever=vector_retriever)
vc_rag = GraphRAG(llm=llm, retriever=vc_retriever)

In [ ]:
q = "How is precision medicine applied to Lupus?"
print(f"\n\nVector Response: \n{v_rag.search(q, retriever_config={'top_k':3}).answer}")
print(f"\n\nVector + Cypher Response: \n{vc_rag.search(q, retriever_config={'top_k':3}).answer}")

In [ ]:
q = "Can you summarize systemic lupus erythematosus (SLE)? including common effects, biomarkers, treatments, and current challenges faced by Physicians and patients? Provide in list format."
print(f"\n\nVector Response: \n{v_rag.search(q, retriever_config={'top_k':3}).answer}")
print(f"\n\nVector + Cypher Response: \n{vc_rag.search(q, retriever_config={'top_k':3}).answer}")